# cs5044 final project experiment 2a: predicting over entire dataset

In [2]:
import math
import pandas as pd
from IPython.display import display
from sklearn import preprocessing
from sklearn import linear_model, neighbors
from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
import numpy as np

# preprocessing

### read in data

In [3]:
df_index = pd.read_csv('sp500_index.csv')
df_stocks = pd.read_csv('sp500_stocks.csv')
df_companies = pd.read_csv('sp500_companies.csv')

### convert to datetime objects

In [4]:
df_stocks['Date'] = pd.to_datetime(df_stocks['Date'], utc=True, errors='coerce')
df_stocks['Date'] = pd.DatetimeIndex(df_stocks['Date']).date

df_index['Date'] = pd.to_datetime(df_index['Date'], utc=True)
df_index['Date'] = pd.DatetimeIndex(df_index['Date']).date

### merge

In [5]:
df = df_companies.merge(df_stocks, on='Symbol', how='inner')
df = df.merge(df_index, on='Date')
df.head()

Exchange Symbol              Shortname               Longname  \
0      NMS   AAPL             Apple Inc.             Apple Inc.   
1      NMS   MSFT  Microsoft Corporation  Microsoft Corporation   
2      NMS  GOOGL          Alphabet Inc.          Alphabet Inc.   
3      NMS   GOOG          Alphabet Inc.          Alphabet Inc.   
4      NMS   AMZN       Amazon.com, Inc.       Amazon.com, Inc.   

                   Sector                        Industry  Currentprice  \
0              Technology            Consumer Electronics        140.42   
1              Technology         Software—Infrastructure        229.25   
2  Communication Services  Internet Content & Information         97.86   
3  Communication Services  Internet Content & Information         98.71   
4       Consumer Cyclical                 Internet Retail        113.67   

       Marketcap        Ebitda  Revenuegrowth  ...  \
0  2256661643264  1.295570e+11          0.019  ...   
1  1721204342784  9.798300e+10          0.124  ...   
2  1281926889472  9.688700e+10          0.126  ...   
3  1281502609408  9.688700e+10          0.126  ...   
4  1158024396800  5.262000e+10          0.072  ...   

                                 Longbusinesssummary    Weight        Date  \
0  Apple Inc. designs, manufactures, and markets ...  0.070093  2012-10-11   
1  Microsoft Corporation develops, licenses, and ...  0.053462  2012-10-11   
2  Alphabet Inc. provides various products and pl...  0.039818  2012-10-11   
3  Alphabet Inc. provides various products and pl...  0.039804  2012-10-11   
4  Amazon.com, Inc. engages in the retail sale of...  0.035969  2012-10-11   

   Adj Close      Close       High        Low       Open       Volume   S&P500  
0  19.372286  22.432142  23.114286  22.432142  23.089287  546081200.0  1432.84  
1  23.988226  28.950001  29.250000  28.870001  29.219999   41488500.0  1432.84  
2  18.805805  18.805805  18.981482  18.776026  18.841341   95260644.0  1432.84  
3  18.716845  18.716845  18.891689  18.687206  18.752213   95713418.0  1432.84  
4  12.211000  12.211000  12.465000  12.094500  12.400000   68946000.0  1432.84  

[5 rows x 24 columns]

### drop irrelevant columns

In [6]:
df.drop(labels=['Shortname', 'Longname', 'City', 'Longbusinesssummary', 'Date'], axis=1, inplace=True)

### one hot encode 

In [7]:
ohe = preprocessing.OneHotEncoder(dtype=int, sparse=False, handle_unknown="ignore")
data = ohe.fit_transform(df[['Exchange', 'Sector', 'Industry', 'State', 'Country']])
cats = pd.DataFrame(data, columns=ohe.get_feature_names())
snp_df = pd.concat([cats, df], axis=1)
snp_df.drop(columns=['Exchange', 'Sector', 'Industry', 'State', 'Country'], inplace=True)

snp_df

/Users/caitlinhermann/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


x0_BTS  x0_NGM  x0_NMS  x0_NYQ  x1_Basic Materials  \
0             0       0       1       0                   0   
1             0       0       1       0                   0   
2             0       0       1       0                   0   
3             0       0       1       0                   0   
4             0       0       1       0                   0   
...         ...     ...     ...     ...                 ...   
1244920       0       0       0       1                   0   
1244921       0       0       1       0                   0   
1244922       0       0       0       1                   0   
1244923       0       0       0       1                   0   
1244924       0       0       0       1                   0   

         x1_Communication Services  x1_Consumer Cyclical  \
0                                0                     0   
1                                0                     0   
2                                1                     0   
3                                1                     0   
4                                0                     1   
...                            ...                   ...   
1244920                          0                     0   
1244921                          0                     0   
1244922                          0                     0   
1244923                          0                     1   
1244924                          0                     0   

         x1_Consumer Defensive  x1_Energy  x1_Financial Services  ...  \
0                            0          0                      0  ...   
1                            0          0                      0  ...   
2                            0          0                      0  ...   
3                            0          0                      0  ...   
4                            0          0                      0  ...   
...                        ...        ...                    ...  ...   
1244920                      0          0                      0  ...   
1244921                      1          0                      0  ...   
1244922                      0          0                      0  ...   
1244923                      0          0                      0  ...   
1244924                      0          0                      0  ...   

         Revenuegrowth  Fulltimeemployees    Weight  Adj Close      Close  \
0                0.019           154000.0  0.070093  19.372286  22.432142   
1                0.124           221000.0  0.053462  23.988226  28.950001   
2                0.126           174014.0  0.039818  18.805805  18.805805   
3                0.126           174014.0  0.039804  18.716845  18.716845   
4                0.072          1523000.0  0.035969  12.211000  12.211000   
...                ...                ...       ...        ...        ...   
1244920          0.041             9300.0  0.000186  23.670000  23.670000   
1244921         -0.065            32000.0  0.000186  14.070000  14.070000   
1244922          0.741            22092.0  0.000162  41.180000  41.180000   
1244923        167.368            34700.0  0.000154  11.910000  11.910000   
1244924          0.163             3224.0  0.000127  21.250000  21.250000   

              High        Low       Open       Volume   S&P500  
0        23.114286  22.432142  23.089287  546081200.0  1432.84  
1        29.250000  28.870001  29.219999   41488500.0  1432.84  
2        18.981482  18.776026  18.841341   95260644.0  1432.84  
3        18.891689  18.687206  18.752213   95713418.0  1432.84  
4        12.465000  12.094500  12.400000   68946000.0  1432.84  
...            ...        ...        ...          ...      ...  
1244920  24.400000  23.650000  24.110001    1255555.0  3612.39  
1244921  14.210000  13.950000  14.020000    2551003.0  3612.39  
1244922  41.910000  40.884998  41.680000     844974.0  3612.39  
1244923  12.985000  11.500000  12.820000   28123604.0  3612.39  
1244924  21.49

### check % of missing values

In [8]:
pd.options.display.max_columns = None
pd.options.display.max_rows = 250
percent_missing = snp_df.isnull().sum() * 100 / len(snp_df)
missing_value_df = pd.DataFrame({'column_name': snp_df.columns,
'percent_missing': percent_missing})

missing_value_df

column_name  \
x0_BTS                                                                        x0_BTS   
x0_NGM                                                                        x0_NGM   
x0_NMS                                                                        x0_NMS   
x0_NYQ                                                                        x0_NYQ   
x1_Basic Materials                                                x1_Basic Materials   
x1_Communication Services                                  x1_Communication Services   
x1_Consumer Cyclical                                            x1_Consumer Cyclical   
x1_Consumer Defensive                                          x1_Consumer Defensive   
x1_Energy                                                                  x1_Energy   
x1_Financial Services                                          x1_Financial Services   
x1_Healthcare                                                          x1_Healthcare   
x1_Industrials                                                        x1_Industrials   
x1_Real Estate                                                        x1_Real Estate   
x1_Technology                                                          x1_Technology   
x1_Utilities                                                            x1_Utilities   
x1_nan                                                                        x1_nan   
x2_Advertising Agencies                                      x2_Advertising Agencies   
x2_Aerospace & Defense                                        x2_Aerospace & Defense   
x2_Agricultural Inputs                                        x2_Agricultural Inputs   
x2_Airlines                                                              x2_Airlines   
x2_Apparel Manufacturing                                    x2_Apparel Manufacturing   
x2_Apparel Retail                                                  x2_Apparel Retail   
x2_Asset Management                                              x2_Asset Management   
x2_Auto & Truck Dealerships                              x2_Auto & Truck Dealerships   
x2_Auto Manufacturers                                          x2_Auto Manufacturers   
x2_Auto Parts                                                          x2_Auto Parts   
x2_Banks—Diversified                                            x2_Banks—Diversified   
x2_Banks—Regional                                                  x2_Banks—Regional   
x2_Beverages—Brewers                                            x2_Beverages—Brewers   
x2_Beverages—Non-Alcoholic                                x2_Beverages—Non-Alcoholic   
x2_Beverages—Wineries & Distilleries            x2_Beverages—Wineries & Distilleries   
x2_Biotechnology                                                    x2_Biotechnology   
x2_Building Materials                                          x2_Building Materials   
x2_Building Products & Equipment                    x2_Building Products & Equipment   
x2_Business Equipment & Supplies                    x2_Business Equipment & Supplies   
x2_Capital Markets                                                x2_Capital Markets   
x2_Chemicals                                                            x2_Chemicals   
x2_Communication Equipment                                x2_Communication Equipment   
x2_Computer Hardware                                            x2_Computer Hardware   
x2_Confectioners                                                    x2_Confectioners   
x2_Conglomerates                                                    x2_Conglomerates   
x2_Consulting Services                                        x2_Consulting Services   
x2_Consumer Electronics                                      x2_Consumer Electronics   
x2_Copper                                                                  x2_Copper   
x2_Credit Services                                                x2_Credit Services   
x2_Diagnostics & Research

### drop missing values

In [9]:
snp_df.dropna(inplace=True)
snp_df

x0_BTS  x0_NGM  x0_NMS  x0_NYQ  x1_Basic Materials  \
0             0       0       1       0                   0   
1             0       0       1       0                   0   
2             0       0       1       0                   0   
3             0       0       1       0                   0   
4             0       0       1       0                   0   
...         ...     ...     ...     ...                 ...   
1244920       0       0       0       1                   0   
1244921       0       0       1       0                   0   
1244922       0       0       0       1                   0   
1244923       0       0       0       1                   0   
1244924       0       0       0       1                   0   

         x1_Communication Services  x1_Consumer Cyclical  \
0                                0                     0   
1                                0                     0   
2                                1                     0   
3                                1                     0   
4                                0                     1   
...                            ...                   ...   
1244920                          0                     0   
1244921                          0                     0   
1244922                          0                     0   
1244923                          0                     1   
1244924                          0                     0   

         x1_Consumer Defensive  x1_Energy  x1_Financial Services  \
0                            0          0                      0   
1                            0          0                      0   
2                            0          0                      0   
3                            0          0                      0   
4                            0          0                      0   
...                        ...        ...                    ...   
1244920                      0          0                      0   
1244921                      1          0                      0   
1244922                      0          0                      0   
1244923                      0          0                      0   
1244924                      0          0                      0   

         x1_Healthcare  x1_Industrials  x1_Real Estate  x1_Technology  \
0                    0               0               0              1   
1                    0               0               0              1   
2                    0               0               0              0   
3                    0               0               0              0   
4                    0               0               0              0   
...                ...             ...             ...            ...   
1244920              1               0               0              0   
1244921              0               0               0              0   
1244922              0               1               0              0   
1244923              0               0               0              0   
1244924              0               0               1              0   

         x1_Utilities  x1_nan  x2_Advertising Agencies  \
0                   0       0                        0   
1                   0       0                        0   
2                   0       0                        0   
3                   0       0                        0   
4                   0       0                        0   
...               ...     ...                      ...   
1244920             0       0                        0   
1244921             0       0                        0   
1244922             0       0                        0   
1244923             0       0                        0   
1244924             0       0                        0   

         x2_Aerospace & Defense  x2_Agricultural Inputs  x2_Airlines  \
0                             0                       0            0

### separate target and features

In [10]:
snp_xvalues = snp_df.drop(["Adj Close", "Close", "High", "Low", "Open", "Symbol"], axis=1)
snp_target = snp_df["Adj Close"]

# linear regression

In [11]:
reg = linear_model.LinearRegression()

kf = model_selection.KFold(n_splits=5, shuffle=False)

r2scores = []
MSEscores = []

for train_index, test_index in kf.split(snp_df[:200000]):
    X_train, X_test = snp_xvalues.iloc[train_index], snp_xvalues.iloc[test_index]
    y_train, y_test = snp_target.iloc[train_index], snp_target.iloc[test_index]
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    r2scores.append(metrics.r2_score(y_test, y_pred))
    MSEscores.append(math.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print("r2=  {}".format(metrics.r2_score(y_test, y_pred)))
    print("root MSE= {}".format(math.sqrt(metrics.mean_squared_error(y_test, y_pred))))
    print()

print("average r2=  {}".format(np.mean(r2scores)))
print("std=  {}".format(np.std(r2scores)))
print("average root MSE=  {}".format(np.mean(MSEscores)))
print("std=  {}".format(np.std(MSEscores)))
print()

print("predicted:")
print(y_pred[-25:])
print("ground truth:")
print(snp_target[-25:].values)

r2=  0.7893249317038847
root MSE= 28.180745530137248

r2=  0.8515775189636916
root MSE= 26.33684092829779

r2=  0.8215455064234788
root MSE= 31.576694823878523

r2=  0.8004877371366774
root MSE= 39.4194956211616

r2=  0.7976324932494948
root MSE= 40.597213747157845

average r2=  0.8121136374954455
std=  0.02240780355810773
average root MSE=  33.2221981301266
std=  5.802230963852303

predicted:
[ 56.79516485  74.58171985  26.29946985  37.25292466  37.17593391
  40.50843293  68.94107603  56.57729372  67.46337527  54.27898443
  15.41891939  35.51744209  67.72144915  68.24909425  32.31447573
  49.09422873  52.07302851  32.14429689  73.12309777  60.87461202
  41.01683027  32.17703857 125.74430433  19.89741026  40.80774393]
ground truth:
[149.57000732  12.05000019  32.84999847  34.06000137  57.02000046
  49.58000183  35.38000107  14.11999989  64.13999939  90.68000031
  89.19000244  61.18999863  41.02000046  26.78000069   6.51000023
  14.43000031  44.83000183  88.63999939  95.22000122  28.079

# kNN regression

### k=1

In [12]:
knn_reg = neighbors.KNeighborsRegressor(n_neighbors=1, weights='uniform')

kf = model_selection.KFold(n_splits=5, shuffle=False)

r2scores = []
MSEscores = []

for train_index, test_index in kf.split(snp_df[:200000]):
    X_train, X_test = snp_xvalues.iloc[train_index], snp_xvalues.iloc[test_index]
    y_train, y_test = snp_target.iloc[train_index], snp_target.iloc[test_index]
    knn_reg.fit(X_train, y_train)
    y_pred = knn_reg.predict(X_test)
    r2scores.append(metrics.r2_score(y_test, y_pred))
    MSEscores.append(math.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print("r2=  {}".format(metrics.r2_score(y_test, y_pred)))
    print("root MSE= {}".format(math.sqrt(metrics.mean_squared_error(y_test, y_pred))))
    print()

print("average r2=  {}".format(np.mean(r2scores)))
print("std=  {}".format(np.std(r2scores)))
print("average root MSE=  {}".format(np.mean(MSEscores)))
print("std=  {}".format(np.std(MSEscores)))
print()

print("predicted:")
print(y_pred[-25:])
print("ground truth:")
print(snp_target[-25:].values)

r2=  0.7600676860479154
root MSE= 30.073936462347838

r2=  0.885151231873816
root MSE= 23.167384567471046

r2=  0.9196463042518905
root MSE= 21.188775224491863

r2=  0.9094133103957457
root MSE= 26.561858601563376

r2=  0.9122443886782642
root MSE= 26.733959495415238

average r2=  0.8773045842495264
std=  0.05975055089266478
average root MSE=  25.545182870257868
std=  3.085081549970384

predicted:
[ 47.22465515  42.68000031  16.08398056  48.04999924  24.4479847
  30.60090828  83.36432648  37.18863678  57.18999863  62.53381348
   7.58427572   9.38951969  48.37307358  31.35686302  16.4005146
  36.96192932  29.89696312  15.88216877  65.11398315  44.84886551
  81.55999756  35.99101639 109.37672424  17.94104576   4.73799992]
ground truth:
[149.57000732  12.05000019  32.84999847  34.06000137  57.02000046
  49.58000183  35.38000107  14.11999989  64.13999939  90.68000031
  89.19000244  61.18999863  41.02000046  26.78000069   6.51000023
  14.43000031  44.83000183  88.63999939  95.22000122  28.0

### k=3

In [13]:
knn_reg = neighbors.KNeighborsRegressor(n_neighbors=3, weights='uniform')

kf = model_selection.KFold(n_splits=5, shuffle=True)

r2scores = []
MSEscores = []

for train_index, test_index in kf.split(snp_df[:200000]):
    X_train, X_test = snp_xvalues.iloc[train_index], snp_xvalues.iloc[test_index]
    y_train, y_test = snp_target.iloc[train_index], snp_target.iloc[test_index]
    knn_reg.fit(X_train, y_train)
    y_pred = knn_reg.predict(X_test)
    r2scores.append(metrics.r2_score(y_test, y_pred))
    MSEscores.append(math.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print("r2=  {}".format(metrics.r2_score(y_test, y_pred)))
    print("root MSE= {}".format(math.sqrt(metrics.mean_squared_error(y_test, y_pred))))
    print()

print("average r2=  {}".format(np.mean(r2scores)))
print("std=  {}".format(np.std(r2scores)))
print("average root MSE=  {}".format(np.mean(MSEscores)))
print("std=  {}".format(np.std(MSEscores)))
print()

print("predicted:")
print(y_pred[-25:])
print("ground truth:")
print(snp_target[-25:].values)

r2=  0.9427013611395375
root MSE= 18.784733645159882

r2=  0.9443816185417975
root MSE= 18.311501486562108

r2=  0.9382904033525331
root MSE= 19.049384901282025

r2=  0.9410795427797757
root MSE= 18.8932592262546

r2=  0.9315406966163172
root MSE= 20.3865087248038

average r2=  0.9395987244859922
std=  0.004501972232453204
average root MSE=  19.085077596812482
std=  0.6957611262993736

predicted:
[ 60.54144287  27.28465907  22.49333318  83.65819041  49.5531311
  49.07721202  36.12414296  57.94444529  68.12260183  39.98721949
  18.45666663 114.69961802  46.77081426  22.78692881 115.83333333
  48.21067556  58.7052714   88.81503805  41.8360494   48.32513173
  49.00666682  26.84973907  33.19983228  52.69674555  40.79783503]
ground truth:
[149.57000732  12.05000019  32.84999847  34.06000137  57.02000046
  49.58000183  35.38000107  14.11999989  64.13999939  90.68000031
  89.19000244  61.18999863  41.02000046  26.78000069   6.51000023
  14.43000031  44.83000183  88.63999939  95.22000122  28.0

### k=5

In [14]:
knn_reg = neighbors.KNeighborsRegressor(n_neighbors=5, weights='uniform')

kf = model_selection.KFold(n_splits=5, shuffle=True)

r2scores = []
MSEscores = []

for train_index, test_index in kf.split(snp_df[:200000]):
    X_train, X_test = snp_xvalues.iloc[train_index], snp_xvalues.iloc[test_index]
    y_train, y_test = snp_target.iloc[train_index], snp_target.iloc[test_index]
    knn_reg.fit(X_train, y_train)
    y_pred = knn_reg.predict(X_test)
    r2scores.append(metrics.r2_score(y_test, y_pred))
    MSEscores.append(math.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print("r2=  {}".format(metrics.r2_score(y_test, y_pred)))
    print("root MSE= {}".format(math.sqrt(metrics.mean_squared_error(y_test, y_pred))))
    print()

print("average r2=  {}".format(np.mean(r2scores)))
print("std=  {}".format(np.std(r2scores)))
print("average root MSE=  {}".format(np.mean(MSEscores)))
print("std=  {}".format(np.std(MSEscores)))
print()

print("predicted:")
print(y_pred[-25:])
print("ground truth:")
print(snp_target[-25:].values)

r2=  0.9489858297955759
root MSE= 17.94075157121918

r2=  0.9457062261450361
root MSE= 17.87713103609962

r2=  0.9488639604147128
root MSE= 17.302629872807533

r2=  0.9434116900650464
root MSE= 18.65341426018508

r2=  0.9444176301051777
root MSE= 18.258313893201056

average r2=  0.9462770673051096
std=  0.0022813663997116426
average root MSE=  18.006448126702498
std=  0.4468558210535994

predicted:
[  16.28154736   45.93080444   98.51600037   50.94958801   30.21787186
   98.61122742   33.92612076 1091.69403076   69.7659996    38.36885986
   41.71073914   32.92899895   41.30576019   26.55982742   41.76724625
   23.07000008   32.50599976   48.44431305   17.23588333   22.27795677
  108.93493347   42.44925537   61.43649673   59.67083817   18.31989288]
ground truth:
[149.57000732  12.05000019  32.84999847  34.06000137  57.02000046
  49.58000183  35.38000107  14.11999989  64.13999939  90.68000031
  89.19000244  61.18999863  41.02000046  26.78000069   6.51000023
  14.43000031  44.83000183  88

# neural network

In [15]:
X = snp_xvalues.to_numpy()
y = snp_target.to_numpy()

## activation='relu'

### layers=2, size=100

In [16]:
kfold = model_selection.KFold(5, shuffle=True, random_state=2)

r2= []
RMSE = []

for train_idx, test_idx in kfold.split(snp_df[:200000]):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    mlp_regr = MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=2000, activation="relu", random_state=2)
    X_scaler = preprocessing.MinMaxScaler()
    
    X_train = X_scaler.fit_transform(X_train)    
    
    mlp_regr.fit(X_train, y_train)
    
    X_test = X_scaler.transform(X_test)
    y_pred = mlp_regr.predict(X_test)
    
    r2 += [metrics.r2_score(y_test, y_pred)]
    RMSE += [math.sqrt(metrics.mean_squared_error(y_test, y_pred))]

print("r2    = {:.4f} ±{:.4f}".format(np.mean(r2), np.std(r2)))
print("RMSE = {:.4f} ±{:.4f}".format(np.mean(RMSE), np.std(RMSE)))

r2    = 0.9961 ±0.0002
RMSE = 4.8785 ±0.2399


## activation='tanh'

### layers=2, size=100

In [17]:
kfold = model_selection.KFold(5, shuffle=True, random_state=2)

r2= []
RMSE = []

for train_idx, test_idx in kfold.split(snp_df[:200000]):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    mlp_regr = MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=2000, activation="tanh", random_state=2)
    X_scaler = preprocessing.MinMaxScaler()
    
    X_train = X_scaler.fit_transform(X_train)    
    
    mlp_regr.fit(X_train, y_train)
    
    X_test = X_scaler.transform(X_test)
    y_pred = mlp_regr.predict(X_test)
    
    r2 += [metrics.r2_score(y_test, y_pred)]
    RMSE += [math.sqrt(metrics.mean_squared_error(y_test, y_pred))]

print("r2    = {:.4f} ±{:.4f}".format(np.mean(r2), np.std(r2)))
print("RMSE = {:.4f} ±{:.4f}".format(np.mean(RMSE), np.std(RMSE)))

r2    = 0.9960 ±0.0004
RMSE = 4.9014 ±0.2659
